In [1]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
import os
import time
import numpy as np
import torch.utils.data as data
from skimage import io, transform, img_as_float

# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 2  # 1 class (car) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
#model_classification = torch.hub.load('pytorch/vision:v0.6.0', 'googlenet', pretrained=True) 
#in_features = model_classification.fc.in_features 
out_features = 2 
#model_classification.fc = nn.Linear(in_features,out_features)
#model_loader = torch.load("./classification_model",map_location=torch.device('cpu')) 
#model_classification.load_state_dict(model_loader['model_state_dict']) 
model.roi_heads.box_predictor.cls_score = None#model_classification #ourmodel here.

In [2]:
device = 'cpu'

In [3]:
def get_transform(train):
    x = []
    x.append(transforms.ToTensor())
    if train:
        x.append(transforms.RandomHorizontalFlip(0.5))
    return transforms.Compose(x)

In [4]:
from PIL import Image 
L = []
for i in range(907):
    try:
        with open('./labels/P'+"{:04d}".format(i)+'.txt', 'r') as file:
            lines=file.read().splitlines()
            img_name = os.path.join('./images/P'+"{:04d}".format(i)+'.png',)
            image = Image.open(img_name).convert("RGB") 
            L.append('P'+"{:04d}".format(i))
            
    except:
        continue
      

In [37]:
class DataSet(object):
    def __init__(self, root_dir, transform, L):
        self.transform = transform
        self.root_dir = root_dir
        self.L = L
    def __getitem__(self, idx):
        
        
        with open('./labels/'+L[idx]+'.txt', 'r') as file:
            lines=file.read().splitlines()
        img_name = os.path.join('./images/'+L[idx]+'.png',)
        image = Image.open(img_name).convert("RGB") 
        XL = []
        for elements in lines[2:]:
            x = elements.split(" ")
            if (x[len(x)-2]=="small-vehicle"):
                XL.append(x)

        # get bounding box coordinates for each mask
        num_objs = len(XL)
        boxes = []
        for i in range(num_objs):
            xmin = float(XL[i][2])
            xmax = float(XL[i][3])
            ymin = float(XL[i][6])
            ymax = float(XL[i][7])
            boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)

        image_id = torch.tensor([idx])
        #print(boxes)
        if (boxes.size()[0]== 0):
            area=None
        
        else:
            area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transform is not None:
            img=self.transform(image)
            target = self.transform(target)
        
        return img, target

    def __len__(self):
        return len(self.L)

In [38]:
dataset = DataSet("./",get_transform(train=True),L)
dataset_test = DataSet("./", get_transform(train=False),L)
indices = torch.randperm(len(dataset)).tolist()
dataset = torch.utils.data.Subset(dataset, indices[:-50])
dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

In [39]:
data_loader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, num_workers=4)

In [40]:
model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params,lr=0.005,momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=3,gamma=0.1)

In [41]:
num_epochs = 10
from engine import train_one_epoch, evaluate
import utils
for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
    lr_scheduler.step()
    evaluate(model, data_loader_test, device=device)

TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/tarhouni/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 198, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/tarhouni/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/tarhouni/anaconda3/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/tarhouni/anaconda3/lib/python3.7/site-packages/torch/utils/data/dataset.py", line 272, in __getitem__
    return self.dataset[self.indices[idx]]
  File "<ipython-input-37-9db034fc3a92>", line 54, in __getitem__
    target = self.transform(target)
  File "/home/tarhouni/anaconda3/lib/python3.7/site-packages/torchvision/transforms/transforms.py", line 67, in __cal

In [31]:
import torch
import torchvision.transforms.functional as F

In [ ]:
import torch
import torchvision.transforms.functional as F
a = torch.FloatTensor(1, height, width)
a = F.to_pil_image(a) 